In [1]:
import numpy as np
from inversion import *
from ray_tracer import *

In [5]:
# -------------------------------------------------------------
# 1.  True 1-D velocity model (unchanged)
# -------------------------------------------------------------
true_velocity_model = [
    {'top': 0.0, 'bottom': 2.0,  'velocity': 2.0},
    {'top': 2.0, 'bottom': 6.0,  'velocity': 3.0},
    {'top': 6.0, 'bottom': 15.0, 'velocity': 3.5},
]

# -------------------------------------------------------------
# 2.  Ten stations, 5 surface + 5 along a borehole (x=y=5 km)
# -------------------------------------------------------------
stations = [
    # ---- surface ring ---------------------------------------------------
    {'x':  0.0, 'y':  0.0, 'z': 0.0},
    {'x': 10.0, 'y':  0.0, 'z': 0.0},
    {'x':  0.0, 'y': 10.0, 'z': 0.0},
    {'x': 10.0, 'y': 10.0, 'z': 0.0},
    {'x':  5.0, 'y':  5.0, 'z': 0.0},   # top of borehole

    # ---- down-hole receivers -------------------------------------------
    {'x':  5.0, 'y':  5.0, 'z': 1.0},   # inside layer 1
    {'x':  5.0, 'y':  5.0, 'z': 3.0},   # crosses into layer 2
    {'x':  5.0, 'y':  5.0, 'z': 5.0},
    {'x':  5.0, 'y':  5.0, 'z': 7.0},   # samples top of layer 3
    {'x':  5.0, 'y':  5.0, 'z': 9.0},   # deepest receiver
]

# -------------------------------------------------------------
# 3.  True event (unchanged)
# -------------------------------------------------------------
true_event = {'x':2.0, 'y': 2.0, 'z': 3.0, 't0': 0.5}

# -------------------------------------------------------------
# 4.  Synthetic arrival times + ray-parameter printout
# -------------------------------------------------------------
true_arrivals = []
for sta in stations:
    t_calc, _, p = trace_ray(true_event, sta, true_velocity_model)
    true_arrivals.append(t_calc + true_event['t0'])
    print(f"Station ({sta['x']:4.1f},{sta['y']:4.1f},{sta['z']:4.1f})  p = {p:7.4f} s/km")

# -------------------------------------------------------------
# 5.  Add Gaussian noise  (σ = 0.01 s)
# -------------------------------------------------------------
np.random.seed(42)
observed_arrivals = [t + np.random.normal(0, 0.01) for t in true_arrivals]

# -------------------------------------------------------------
# 6.  Initial guesses (slightly off)
# -------------------------------------------------------------
initial_event = {'x': 3.0, 'y': 3.0, 'z': 2.5, 't0': 0.4}

initial_velocity_model = [
    {'top': 0.0, 'bottom': 2.0,  'velocity': 2.5},
    {'top': 2.0, 'bottom': 6.0,  'velocity': 3.5},
    {'top': 6.0, 'bottom': 15.0, 'velocity': 4.0},
]

# -------------------------------------------------------------
# 7.  Run the joint inversion
# -------------------------------------------------------------
updated_event, updated_model, _ = invert_event_and_velocity(
    event=initial_event,
    stations=stations,
    picks=observed_arrivals,
    velocity_model=initial_velocity_model,
    max_iter=2000,
    tol=1e-5,        
    damp_xyz= 1e-5,
    damp_t0= 1e-5,
    damp_slow= 1e-3,
)

# -------------------------------------------------------------
# 8.  Report results
# -------------------------------------------------------------
print("\n=== True Event ===")
print(true_event)

print("\n=== Estimated Event ===")
print(updated_event)

print("\n=== Updated Velocity Model ===")
for layer in updated_model:
    print(layer)


Station ( 0.0, 0.0, 0.0)  p =  0.2772 s/km
Station (10.0, 0.0, 0.0)  p =  0.3294 s/km
Station ( 0.0,10.0, 0.0)  p =  0.3294 s/km
Station (10.0,10.0, 0.0)  p =  0.3315 s/km
Station ( 5.0, 5.0, 0.0)  p =  0.3118 s/km
Station ( 5.0, 5.0, 1.0)  p =  0.3199 s/km
Station ( 5.0, 5.0, 3.0)  p =  0.1910 s/km
Station ( 5.0, 5.0, 5.0)  p =  0.3015 s/km
Station ( 5.0, 5.0, 7.0)  p =  0.2306 s/km
Station ( 5.0, 5.0, 9.0)  p =  0.1769 s/km
Iter 01  RMS=0.7696  |Δm|=2.34e+00
Iter 02  RMS=1.7169  |Δm|=3.00e+00
Iter 03  RMS=1.0960  |Δm|=5.24e+00
Iter 04  RMS=0.8421  |Δm|=3.12e+00
Iter 05  RMS=0.8988  |Δm|=2.88e+00
Iter 06  RMS=0.6221  |Δm|=2.83e+00
Iter 07  RMS=1.0843  |Δm|=3.14e+00
Iter 08  RMS=3.5116  |Δm|=2.75e+00
Iter 09  RMS=0.8229  |Δm|=1.60e+00
Iter 10  RMS=0.8659  |Δm|=6.17e-01
Iter 11  RMS=0.4287  |Δm|=1.07e+00
Iter 12  RMS=1.1154  |Δm|=8.76e-01
Iter 13  RMS=0.4786  |Δm|=4.38e-01
Iter 14  RMS=0.4014  |Δm|=3.88e-01
Iter 15  RMS=0.4141  |Δm|=5.67e-01
Iter 16  RMS=0.3866  |Δm|=5.86e-01
Iter 17  R

Iter 224  RMS=0.0049  |Δm|=4.47e-03
Iter 225  RMS=0.0564  |Δm|=5.01e-03
Iter 226  RMS=0.0072  |Δm|=5.07e-01
Iter 227  RMS=0.0644  |Δm|=5.95e-01
Iter 228  RMS=0.0211  |Δm|=9.82e-02
Iter 229  RMS=0.0096  |Δm|=1.53e-02
Iter 230  RMS=0.0093  |Δm|=1.10e-02
Iter 231  RMS=0.0056  |Δm|=1.32e-02
Iter 232  RMS=0.0084  |Δm|=2.10e-02
Iter 233  RMS=0.0045  |Δm|=5.00e-03
Iter 234  RMS=0.0154  |Δm|=1.21e-01
Iter 235  RMS=0.0190  |Δm|=1.50e-01
Iter 236  RMS=0.0840  |Δm|=1.60e-02
Iter 237  RMS=0.0104  |Δm|=3.20e-02
Iter 238  RMS=0.0097  |Δm|=7.62e-03
Iter 239  RMS=0.0121  |Δm|=3.32e-02
Iter 240  RMS=0.0075  |Δm|=2.86e-03
Iter 241  RMS=0.0102  |Δm|=6.96e-03
Iter 242  RMS=0.0044  |Δm|=5.03e-03
Iter 243  RMS=0.0052  |Δm|=7.43e-03
Iter 244  RMS=0.0050  |Δm|=1.70e-02
Iter 245  RMS=0.0047  |Δm|=9.55e-03
Iter 246  RMS=0.0045  |Δm|=9.88e-03
Iter 247  RMS=0.0057  |Δm|=1.77e-02
Iter 248  RMS=0.0095  |Δm|=1.07e-02
Iter 249  RMS=0.0087  |Δm|=6.51e-02
Iter 250  RMS=0.0096  |Δm|=6.96e-02
Iter 251  RMS=0.0055  |Δm|=2

Iter 467  RMS=0.0082  |Δm|=1.81e-02
Iter 468  RMS=0.0259  |Δm|=2.96e-02
Iter 469  RMS=0.0086  |Δm|=1.91e-02
Iter 470  RMS=0.0068  |Δm|=1.30e-02
Iter 471  RMS=0.0175  |Δm|=2.35e-02
Iter 472  RMS=0.0082  |Δm|=2.32e-02
Iter 473  RMS=0.0068  |Δm|=3.02e-02
Iter 474  RMS=0.0066  |Δm|=8.22e-03
Iter 475  RMS=0.0051  |Δm|=7.66e-03
Iter 476  RMS=0.0060  |Δm|=6.43e-03
Iter 477  RMS=0.0322  |Δm|=6.15e-03
Iter 478  RMS=0.0069  |Δm|=1.90e-02
Iter 479  RMS=0.0096  |Δm|=4.78e-02
Iter 480  RMS=0.0085  |Δm|=3.01e-02
Iter 481  RMS=0.0072  |Δm|=8.52e-03
Iter 482  RMS=0.0178  |Δm|=9.94e-03
Iter 483  RMS=0.0040  |Δm|=5.68e-03
Iter 484  RMS=0.0092  |Δm|=3.19e-02
Iter 485  RMS=0.0069  |Δm|=3.28e-02
Iter 486  RMS=0.0117  |Δm|=2.90e-02
Iter 487  RMS=0.0088  |Δm|=1.10e-02
Iter 488  RMS=0.0247  |Δm|=1.65e-02
Iter 489  RMS=0.0094  |Δm|=6.95e-03
Iter 490  RMS=0.0057  |Δm|=9.09e-03
Iter 491  RMS=0.0053  |Δm|=2.56e-02
Iter 492  RMS=0.0071  |Δm|=2.97e-02
Iter 493  RMS=0.0065  |Δm|=4.42e-03
Iter 494  RMS=0.0080  |Δm|=2

Iter 696  RMS=0.0124  |Δm|=4.37e-02
Iter 697  RMS=0.0594  |Δm|=4.05e-02
Iter 698  RMS=0.0097  |Δm|=1.16e-02
Iter 699  RMS=0.0096  |Δm|=9.54e-03
Iter 700  RMS=0.0646  |Δm|=4.23e-03
Iter 701  RMS=0.0069  |Δm|=3.97e-03
Iter 702  RMS=0.0065  |Δm|=3.47e-02
Iter 703  RMS=0.0060  |Δm|=3.69e-02
Iter 704  RMS=0.0082  |Δm|=6.95e-03
Iter 705  RMS=0.0057  |Δm|=1.22e-02
Iter 706  RMS=0.0146  |Δm|=1.74e-02
Iter 707  RMS=0.0085  |Δm|=4.29e-02
Iter 708  RMS=0.0087  |Δm|=5.21e-02
Iter 709  RMS=0.0071  |Δm|=2.02e-02
Iter 710  RMS=0.0150  |Δm|=4.42e-02
Iter 711  RMS=0.0112  |Δm|=2.82e-02
Iter 712  RMS=0.0067  |Δm|=5.50e-02
Iter 713  RMS=0.0092  |Δm|=2.75e-02
Iter 714  RMS=0.0060  |Δm|=8.98e-02
Iter 715  RMS=0.0143  |Δm|=3.37e-02
Iter 716  RMS=0.0180  |Δm|=9.98e-02
Iter 717  RMS=0.0055  |Δm|=1.55e-02
Iter 718  RMS=0.0110  |Δm|=2.53e-02
Iter 719  RMS=0.0069  |Δm|=1.01e-02
Iter 720  RMS=0.0056  |Δm|=5.11e-03
Iter 721  RMS=0.0156  |Δm|=3.12e-02
Iter 722  RMS=0.0072  |Δm|=1.40e-02
Iter 723  RMS=0.0083  |Δm|=2

Iter 937  RMS=0.0050  |Δm|=9.29e-03
Iter 938  RMS=0.0091  |Δm|=2.21e-02
Iter 939  RMS=0.0070  |Δm|=4.69e-03
Iter 940  RMS=0.0055  |Δm|=8.87e-03
Iter 941  RMS=0.0089  |Δm|=1.34e-02
Iter 942  RMS=0.0373  |Δm|=1.27e-02
Iter 943  RMS=0.0140  |Δm|=3.03e-02
Iter 944  RMS=0.0086  |Δm|=9.19e-03
Iter 945  RMS=0.0058  |Δm|=1.21e-02
Iter 946  RMS=0.0068  |Δm|=6.15e-03
Iter 947  RMS=0.0059  |Δm|=1.94e-02
Iter 948  RMS=0.0059  |Δm|=1.32e-02
Iter 949  RMS=0.0068  |Δm|=1.21e-02
Iter 950  RMS=0.0067  |Δm|=2.00e-02
Iter 951  RMS=0.0089  |Δm|=2.64e-02
Iter 952  RMS=0.0075  |Δm|=2.62e-02
Iter 953  RMS=0.0106  |Δm|=1.95e-02
Iter 954  RMS=0.0079  |Δm|=1.53e-02
Iter 955  RMS=0.0053  |Δm|=6.95e-03
Iter 956  RMS=0.0043  |Δm|=1.69e-02
Iter 957  RMS=0.0052  |Δm|=1.21e-02
Iter 958  RMS=0.0077  |Δm|=2.49e-02
Iter 959  RMS=0.0057  |Δm|=1.28e-02
Iter 960  RMS=0.0061  |Δm|=3.72e-02
Iter 961  RMS=0.0076  |Δm|=3.13e-02
Iter 962  RMS=0.0092  |Δm|=2.36e-02
Iter 963  RMS=0.0053  |Δm|=1.63e-02
Iter 964  RMS=0.0062  |Δm|=1

Iter 1170  RMS=0.2554  |Δm|=1.46e-02
Iter 1171  RMS=0.0067  |Δm|=1.91e-02
Iter 1172  RMS=0.0046  |Δm|=6.06e-03
Iter 1173  RMS=0.0060  |Δm|=1.17e-02
Iter 1174  RMS=0.0054  |Δm|=3.38e-03
Iter 1175  RMS=0.0090  |Δm|=2.08e-02
Iter 1176  RMS=0.0186  |Δm|=1.48e-02
Iter 1177  RMS=0.0079  |Δm|=2.48e-02
Iter 1178  RMS=0.0081  |Δm|=1.17e-02
Iter 1179  RMS=0.0057  |Δm|=8.48e-03
Iter 1180  RMS=0.0058  |Δm|=2.57e-02
Iter 1181  RMS=0.0078  |Δm|=2.84e-02
Iter 1182  RMS=0.0070  |Δm|=5.23e-03
Iter 1183  RMS=0.0076  |Δm|=1.77e-02
Iter 1184  RMS=0.0073  |Δm|=1.90e-02
Iter 1185  RMS=0.0393  |Δm|=1.83e-02
Iter 1186  RMS=0.0100  |Δm|=3.28e-02
Iter 1187  RMS=0.0099  |Δm|=9.51e-02
Iter 1188  RMS=0.0260  |Δm|=9.91e-02
Iter 1189  RMS=0.0081  |Δm|=1.54e-02
Iter 1190  RMS=0.0109  |Δm|=1.36e-02
Iter 1191  RMS=0.0066  |Δm|=4.94e-02
Iter 1192  RMS=0.0112  |Δm|=5.44e-02
Iter 1193  RMS=0.0050  |Δm|=2.16e-02
Iter 1194  RMS=0.0052  |Δm|=4.88e-03
Iter 1195  RMS=0.0108  |Δm|=6.21e-02
Iter 1196  RMS=0.0190  |Δm|=4.10e-02
I

Iter 1409  RMS=0.0074  |Δm|=1.66e-02
Iter 1410  RMS=0.0052  |Δm|=7.67e-03
Iter 1411  RMS=0.0091  |Δm|=1.87e-02
Iter 1412  RMS=0.0078  |Δm|=3.84e-02
Iter 1413  RMS=0.0094  |Δm|=4.38e-02
Iter 1414  RMS=0.0067  |Δm|=1.95e-02
Iter 1415  RMS=0.0069  |Δm|=1.09e-02
Iter 1416  RMS=0.0055  |Δm|=1.70e-02
Iter 1417  RMS=0.0066  |Δm|=3.23e-02
Iter 1418  RMS=0.0063  |Δm|=1.83e-02
Iter 1419  RMS=0.0058  |Δm|=9.87e-03
Iter 1420  RMS=0.0063  |Δm|=6.17e-03
Iter 1421  RMS=0.0066  |Δm|=5.18e-02
Iter 1422  RMS=0.0108  |Δm|=8.68e-02
Iter 1423  RMS=0.0067  |Δm|=2.85e-02
Iter 1424  RMS=0.0156  |Δm|=3.59e-02
Iter 1425  RMS=0.0083  |Δm|=2.76e-02
Iter 1426  RMS=0.0067  |Δm|=7.54e-03
Iter 1427  RMS=0.0084  |Δm|=3.05e-02
Iter 1428  RMS=0.0103  |Δm|=2.64e-02
Iter 1429  RMS=0.0096  |Δm|=3.29e-03
Iter 1430  RMS=0.0225  |Δm|=2.22e-02
Iter 1431  RMS=0.0057  |Δm|=8.89e-03
Iter 1432  RMS=0.0069  |Δm|=3.04e-03
Iter 1433  RMS=0.0640  |Δm|=9.37e-03
Iter 1434  RMS=0.0098  |Δm|=8.19e-03
Iter 1435  RMS=0.0109  |Δm|=1.82e-02
I

Iter 1648  RMS=0.0054  |Δm|=4.73e-03
Iter 1649  RMS=0.0049  |Δm|=1.39e-02
Iter 1650  RMS=0.0066  |Δm|=1.14e-02
Iter 1651  RMS=0.0109  |Δm|=2.70e-02
Iter 1652  RMS=0.0066  |Δm|=2.72e-02
Iter 1653  RMS=0.0062  |Δm|=1.50e-02
Iter 1654  RMS=0.0057  |Δm|=1.93e-02
Iter 1655  RMS=0.0062  |Δm|=1.48e-02
Iter 1656  RMS=0.0067  |Δm|=1.18e-02
Iter 1657  RMS=0.0070  |Δm|=3.41e-02
Iter 1658  RMS=0.0082  |Δm|=3.83e-02
Iter 1659  RMS=0.0114  |Δm|=2.49e-02
Iter 1660  RMS=0.0080  |Δm|=3.26e-02
Iter 1661  RMS=0.0073  |Δm|=2.61e-02
Iter 1662  RMS=0.0104  |Δm|=4.78e-02
Iter 1663  RMS=0.0101  |Δm|=1.43e-02
Iter 1664  RMS=0.0079  |Δm|=1.89e-02
Iter 1665  RMS=0.0066  |Δm|=9.06e-03
Iter 1666  RMS=0.0056  |Δm|=4.46e-03
Iter 1667  RMS=0.0088  |Δm|=1.41e-02
Iter 1668  RMS=0.0085  |Δm|=7.74e-03
Iter 1669  RMS=0.0072  |Δm|=1.28e-02
Iter 1670  RMS=0.0094  |Δm|=1.56e-02
Iter 1671  RMS=0.0093  |Δm|=1.20e-02
Iter 1672  RMS=0.0062  |Δm|=2.82e-02
Iter 1673  RMS=0.0048  |Δm|=8.76e-03
Iter 1674  RMS=0.0045  |Δm|=6.42e-03
I

Iter 1872  RMS=0.0098  |Δm|=1.16e-02
Iter 1873  RMS=0.0066  |Δm|=5.43e-02
Iter 1874  RMS=0.0106  |Δm|=4.75e-03
Iter 1875  RMS=0.0134  |Δm|=7.10e-02
Iter 1876  RMS=0.0935  |Δm|=1.36e-02
Iter 1877  RMS=0.0107  |Δm|=2.36e-02
Iter 1878  RMS=0.0105  |Δm|=6.39e-02
Iter 1879  RMS=0.0123  |Δm|=1.28e-01
Iter 1880  RMS=0.0361  |Δm|=5.98e-02
Iter 1881  RMS=0.0083  |Δm|=2.43e-02
Iter 1882  RMS=0.0071  |Δm|=1.41e-02
Iter 1883  RMS=0.0072  |Δm|=2.06e-02
Iter 1884  RMS=0.0066  |Δm|=4.82e-03
Iter 1885  RMS=0.0078  |Δm|=2.54e-02
Iter 1886  RMS=0.0103  |Δm|=1.86e-02
Iter 1887  RMS=0.0075  |Δm|=1.76e-02
Iter 1888  RMS=0.0064  |Δm|=2.89e-03
Iter 1889  RMS=0.0545  |Δm|=1.09e-02
Iter 1890  RMS=0.0080  |Δm|=1.13e-02
Iter 1891  RMS=0.0068  |Δm|=5.92e-03
Iter 1892  RMS=0.0064  |Δm|=1.14e-02
Iter 1893  RMS=0.0036  |Δm|=5.25e-03
Iter 1894  RMS=0.0062  |Δm|=1.95e-02
Iter 1895  RMS=0.0069  |Δm|=7.73e-03
Iter 1896  RMS=0.0075  |Δm|=1.22e-02
Iter 1897  RMS=0.0072  |Δm|=7.38e-03
Iter 1898  RMS=0.0052  |Δm|=2.30e-02
I